In [5]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import numpy as np
import matplotlib.pyplot as plt

In [34]:
target_number = int(input("Enter a number between 0 and 3: "))

# Manually convert decimal to binary using a mathematical method
def decimal_to_binary(n, bits=2):
    binary = []
    while n > 0:
        binary.append(str(n % 2))  # Append the remainder (either 0 or 1)
        n //= 2  # Integer division by 2
    # Add leading zeros if the binary number has fewer than `bits` digits
    while len(binary) < bits:
        binary.append('0')
    return ''.join(binary[::-1])  # Reverse the list to get correct order

# Convert the input number to a 2-bit binary string
target_state = decimal_to_binary(target_number, bits=2)
print(f"Target state: |{target_state}⟩")

def oracle(n, target_state):
    qc = QuantumCircuit(n)
    
    # Apply X gates to flip target state to |11⟩ before applying CZ
    for i in range(n):
        if target_state[i] == '0':
            qc.x(i)

    qc.cz(0, 1)  # Phase flip on |11⟩

    # Undo the X gates
    for i in range(n):
        if target_state[i] == '0':
            qc.x(i)

    return qc

def diffusion_operator(n, qc):
    # Apply the diffusion operator (inversion about the mean)
    # Apply Hadamard to all qubits
    qc.h(range(n))
    # Apply X to all qubits
    qc.x(range(n))
    # Apply Hadamard to the last qubit
    qc.h(n-1)
    # Apply CNOT (controlled-X) between the first and second qubit
    qc.cx(0, 1)
    # Apply Hadamard to the last qubit
    qc.h(n-1)
    # Undo the X and Hadamard gates
    qc.x(range(n))
    qc.h(range(n))

    return qc

n = 2
qc = QuantumCircuit(n, n)

# Step 1: Apply Hadamard to all qubits (superposition)
qc.h(range(n))

# Step 2: Apply Grover Iterations
iterations = int(np.pi / 4 * np.sqrt(2**n))  # Optimal number of iterations for n=2 qubits
print(f"Number of Grover iterations: {iterations}")

for _ in range(iterations):
    qc.append(oracle(n, target_state), range(n))  # Apply oracle

    # Apply the diffusion operator
    qc = diffusion_operator(n, qc)

# Step 3: Measurement
qc.measure(range(n), range(n))

# Simulate
simulator = AerSimulator()
transpiled_qc = transpile(qc, simulator)
job = simulator.run(transpiled_qc, shots=1024)
result = job.result()
counts = result.get_counts()

# Display results
print("Grover's Algorithm Results:")
print(counts)

# Manually map binary states to decimal numbers (0-3)
state_to_decimal = {
    '00': 0,  # Binary '00' to Decimal 0
    '01': 1,  # Binary '01' to Decimal 1
    '10': 2,  # Binary '10' to Decimal 2
    '11': 3   # Binary '11' to Decimal 3
}

# Convert binary states back to decimal using manual mapping
converted_counts = {}
for state, count in counts.items():
    decimal_state = state_to_decimal[state]  # Manually map the state
    converted_counts[decimal_state] = count

# Display converted results (manual decimal mapping)
print("Converted Results (0-3):")
print(converted_counts)

Target state: |10⟩
Number of Grover iterations: 1
Grover's Algorithm Results:
{'01': 1024}
Converted Results (0-3):
{1: 1024}
